In [ ]:
# -*- coding: utf-8 -*-
import os
import sys

from io import StringIO

import pandas as pd
from dotenv import load_dotenv

from api_functions import TelegramBot
from api_functions import YandexDirect

### Загрузка параметров окружения

In [ ]:
load_dotenv()

### Telegram Bot Token, chat ID and instance

In [ ]:
bot_token = os.getenv("TELEGRAM_API_TOKEN")
# chat_id = '126841573' # personal
chat_id = '-4090186402' # Мой канал с алёртами
# chat_id = '-1002145547826' # Чат закупки в телеге
telegram_bot = TelegramBot(bot_token, chat_id)

### Параметры для Яндекса

In [ ]:
# OAuth-токен представителя агентства, от имени которого будут выполняться запросы
token = os.getenv("YANDEX_TOKEN")

# Адрес сервиса Reports для отправки JSON-запросов (регистрозависимый)
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

### Метод для корректной обработки строк в кодировке UTF-8 как в Python 3, так и в Python 2

In [ ]:
if sys.version_info < (3,):
    def u(x):
        try:
            return x.encode("utf8")
        except UnicodeDecodeError:
            return x
else:
    def u(x):
        if isinstance(x, bytes):
            return x.decode('utf8')
        else:
            return x

## Запросы в Яндекс

### Create Instance of Yandex Exporter

In [ ]:
yandex = YandexDirect(token)

In [ ]:
logins_actual = ['lost-ark-mrt', 'perfect-world-mrt', 'arche-age-mrt', 
                 'allodsonline-games', 'atomicheart-game', 'battle-teams2', 
                 'warface2016-2017warfacerwyndxg', 'revival-astrum']

### Запрос средств на аккаунтах

In [ ]:
df = pd.DataFrame(yandex.accounts_budget(logins_actual))

### Запрос трат за последние три дня по каждому аккаунту

In [ ]:
budget_dict = yandex.get_account_spent(logins_actual)

### Преобразование временного файла из запроса в pandas df

In [ ]:
budget_dict = pd.read_csv(StringIO(budget_dict), sep=',', escapechar='\\', index_col=False)

### Фильтруем аккаунты только со спендом за последние три дня

In [ ]:
budget_dict = budget_dict[(budget_dict['Costs']!= 0)]

In [ ]:
merged_df = pd.merge(budget_dict, df, how='left', on='Login').fillna(0)

In [ ]:
merged_df['avg_cost'] = round(merged_df['Costs'] / 3, 2)
merged_df['days'] = pd.to_numeric(merged_df['Amount'] // merged_df['avg_cost'], downcast='integer')

## Отправка сообщений в телеграм

### Вариант с одним сообщением

In [ ]:
tg_message = []

Дальше собираем список для сообщения

In [ ]:
for row in merged_df.index:
    if merged_df['days'][row] == 0 and merged_df['Amount'][row] < 5000:
        tg_message.append(f"На аккаунте {merged_df['Login'][row]} закончился бюджет.")
    elif merged_df['days'][row] == 0 and merged_df['Amount'][row] > 5000:
        tg_message.append(f"На аккаунте {merged_df['Login'][row]} бюджета меньше чем на день.")
    elif merged_df['days'][row] == 1:
        tg_message.append(f"{merged_df['Login'][row]} хватит бюджета на {merged_df['days'][row]} день.")
    elif 1 < merged_df['days'][row] < 4:
        tg_message.append(f"{merged_df['Login'][row]} хватит бюджета на {merged_df['days'][row]} дня.")

Преобразуем список в строку с разделителями (каждая запись с новой строки + одна строка отступа)

In [ ]:
tg_message = "\n\n".join(tg_message)

In [ ]:
print(tg_message)

In [ ]:
telegram_bot = TelegramBot(bot_token, chat_id)
telegram_bot.send_message(tg_message)

### Алёрта об срабатывании скрипта

In [ ]:
chat_id = '126841573'
telegram_bot = TelegramBot(bot_token, chat_id)
message_test = f"Yandex отработал"
telegram_bot.send_message(message_test)